In [13]:
'''
The purpose of this function is extract and process an XML file.

Not entirely sure what an XML file is, or how it compares to a JSON file but it seems
very low memory and has a tree structure.

This is how facilities provide reports to the EPA emissions entity. 

Please note that you need to:

'pip install elementpath' and then import the module into your env

to import: 
'xml.etree.ElementTree as ET'

'''

"\nThe purpose of this function is extract and process an XML file.\n\nNot entirely sure what an XML file is, or how it compares to a JSON file but it seems\nvery low memory and has a tree structure.\n\nThis is how facilities provide reports to the EPA emissions entity. \n\nPlease note that you need to:\n\n'pip install elementpath' and then import the module into your env\n\nto import: \n'xml.etree.ElementTree as ET'\n\n"

In [20]:
import pandas as pd
import requests
import xmltodict
import prettyprinter as pp
import json

# used to parse through xml files
import xml
from bs4 import BeautifulSoup
from xml.etree import ElementTree as ET


In [16]:
def meander_xml(ghgp_id, topic, year=2021):

    '''
    Given a ghgp_id, return the fuel type within the xml code
    '''

    try:

        # here, we will be adding the ghgp_id value to a url used to make GET request
        url = f'https://ghgdata.epa.gov/ghgp/service/xml/{year}?id={ghgp_id}&et=undefined'

        # making the url GET request for the xml file
        response = requests.get(url=url)
        webpage_content = response.text

        # modifying our url contnet because it containts code we do not need
        lines = webpage_content[38:-6].split('\n')
        modified_content = '\n'.join(lines[1:])

        # parse the XML
        root = ET.fromstring(modified_content)

        # now we want to create an ifelse statement that will grab specific data
        # user needs

        if topic == 'Fuel Type' or topic == 'fuel type':

            # creating a list to return the found elements to
            fuel_types = []

            # extract the fuel type element
            for fuel_type in root.findall(".//FuelType"):

                # return the fuel_type value
                fuel_types.append(fuel_type.text)
            
            # removing redundancies
            fuel_types = list(set(fuel_types))

            # concatenating our list to output a long string
            fuel_typez = ", ".join([str(item) for item in fuel_types])

            return fuel_typez
    

        elif topic == 'Unit Type' or topic == 'unit type':
    
            # creating a list to return the found elements to
            unit_types = []

            # extract the unit type value
            for unit_type in root.findall('.//UnitType'):

                # return the unit_type value
                unit_types.append(unit_type.text)
            
            # removing redundancies
            unit_types = list(set(unit_types))

            # concatenating our list to output a long string
            unit_typez = ", ".join([str(item) for item in unit_types])

            return unit_typez
    

        elif topic =='Parent Company' or topic == 'parent company': 
        
            # creating a list to return the found elements to
            parent_cos = []
            
            # else, extract parent company name 
            for parent_co in root.findall('.//ParentCompanyLegalName'):

                # return the parent company name
                parent_cos.append(parent_co.text)
            
            # removing redundancies
            parent_cos = list(set(parent_cos))

            # concatenating our list to output a long string
            parent_coz = ", ".join([str(item) for item in parent_cos])

            return parent_coz
    

        else: 

            # means none of the 3 topics were given 
            print('Error: please insert a valid topic to receive a valid value!')


        # now we add this fuel type value to a new column called "Fuel Type"
        # dataframe['Fuel Type'] = str(fuel_type)


    except xml.etree.ElementTree.ParseError:
        # xml file is not properly formatted, so skip this iteration
        "sorry, bud!"
        pass

In [17]:
x = meander_xml(ghgp_id='1000340', year=2021, topic='fuel type')
print(x, '\n')

y = meander_xml(ghgp_id='1000340', year=2021, topic='unit type')
print(y, '\n')

z = meander_xml(ghgp_id='1000340', year=2021, topic='parent company')
print(z, '\n')

print(x, y, z)

Natural Gas (Weighted U.S. Average), Subbituminous 

Electricity Generator, PRH (Process Heater) 

DYNEGY INC, Vistra Corp 

Natural Gas (Weighted U.S. Average), Subbituminous Electricity Generator, PRH (Process Heater) DYNEGY INC, Vistra Corp


This section is dedicated towards making meander_xml more efficienct. 
We want to make sure that for each id, we only make on request of xml data 
and from there we grab whatever it is that we want to grab. 

In [18]:
# year input into the url
year = '2022'

# ghgp id input into the url 
ghgp_id = '1002565'


# url where we will be making our request
url = f'https://ghgdata.epa.gov/ghgp/service/xml/{year}?id={ghgp_id}&et=undefined'

print(url)

https://ghgdata.epa.gov/ghgp/service/xml/2022?id=1002565&et=undefined


In [63]:
# making the url get request for the xml file
response = requests.get(url=url)
content = response.text

# reviewing our data now
bs_data = BeautifulSoup(content, 'xml')

# now we want to search through xml for specific data
finds = ['UnitType', 'UnitName','ParentCompanyLegalName', 'FuelType']

# iterating through and grabbing different data
for element in finds:
    # creating an empty list where we will be adding the values that we are getting
    empty_list = []

    # finding all instances of the given element
    b_finds = bs_data.find_all(element)

    # iterating through list of grabbed values and only grabbing text
    for b_find in b_finds:
        empty_list.append(b_find.text)

    # converting data into a list
    # output_list = set(empty_list)

    print(empty_list)





['PRH (Process Heater)', 'OCS (Other combustion source)', 'PRH (Process Heater)', 'OB (Boiler, other)', 'PRH (Process Heater)', 'OCS (Other combustion source)', 'OB (Boiler, other)', 'TODF (Thermal oxidizer, direct fired, no heat recovery)', 'OB (Boiler, other)', 'PRH (Process Heater)', 'Hydrogen production process unit', 'Flare', 'Sulfur Recovery Plant']
['H-1 Heater OFG', 'GP-Refinery Fuel Gas', 'VH-4 Vacuum Gas', 'Wicks Oilfield Gas', 'H-602 Vacuum Gas', 'GP-Natural Gas Combustion', 'Wicks Boiler Natural Gas', 'Zeeco Thermal Oxidizer Tailgas', 'Unit 4 Standby Boiler', 'Crude Heater Oilfield Gas', 'Hydrogen Unit', 'Flare', 'SRU']
['SAN JOAQUIN REFINING CO INC']
['Fuel Gas', 'Natural Gas (Weighted U.S. Average)', 'Natural Gas (Weighted U.S. Average)', 'Natural Gas (Weighted U.S. Average)']
